In [ ]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Dict, List, Optional

import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

ARTIFACT_ROOT = Path("../../artifacts/experiments").resolve()
plt.style.use("ggplot")

In [ ]:
def _load_json(path: Path) -> Dict:
    try:
        return json.loads(path.read_text(encoding="utf-8"))
    except (FileNotFoundError, json.JSONDecodeError):
        return {}

def _iter_artifacts_sorted() -> List[Path]:
    if not ARTIFACT_ROOT.exists():
        return []
    entries = [p for p in ARTIFACT_ROOT.iterdir() if p.is_dir()]
    entries.sort(key=lambda path: path.stat().st_mtime, reverse=True)
    return entries

def list_artifacts(suite: Optional[str] = None, limit: int = 10) -> List[Path]:
    matches: List[Path] = []
    for path in _iter_artifacts_sorted():
        plan = _load_json(path / "plan.json")
        if suite:
            suite_tag = plan.get("overrides", {}).get("annotations", {}).get("suite")
            if suite_tag != suite:
                continue
        matches.append(path)
        if len(matches) >= limit:
            break
    return matches

def _flatten_metrics(metrics: Dict) -> Dict:
    flat: Dict[str, float] = {}
    for key, value in (metrics or {}).items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flat[f"{key}_{sub_key}"] = sub_value
        else:
            flat[key] = value
    return flat

def collect_run_summary(path: Path):
    plan = _load_json(path / "plan.json")
    result = _load_json(path / "run_result.json")
    rows: List[Dict] = []
    mode = plan.get("mode") or result.get("mode")
    for command in result.get("commands", []):
        metrics = _flatten_metrics(command.get("metrics"))
        if not metrics:
            continue
        row = {"artifact": path.name, "mode": mode, "command": command.get("name"), "role": command.get("role") }
        row.update(metrics)
        rows.append(row)
    frame = pd.DataFrame(rows) if rows else pd.DataFrame()
    return plan, result, frame

In [ ]:
def summarize_suite(suite_name: str, limit: int = 12) -> pd.DataFrame:
    frames: List[pd.DataFrame] = []
    for path in list_artifacts(suite_name, limit=limit):
        plan, result, frame = collect_run_summary(path)
        if frame.empty:
            continue
        enriched = frame.copy()
        enriched["suite"] = suite_name
        enriched["workload"] = plan.get("workload")
        enriched["artifact_path"] = str(path)
        frames.append(enriched)
    if not frames:
        return pd.DataFrame()
    return pd.concat(frames, ignore_index=True)

def plot_throughput(df: pd.DataFrame, command: str, title: str) -> None:
    subset = df[df.get("command") == command]
    if subset.empty:
        print(f"No metrics for {command}.")
        return
    if "throughput_ops_per_s" not in subset.columns:
        print("Throughput column missing; skipping plot.")
        return
    pivot = subset.groupby(["mode"])['throughput_ops_per_s'].mean().sort_index()
    ax = pivot.plot.bar(figsize=(6, 4), title=title)
    ax.set_ylabel("ops/s")
    plt.show()

def preview_metrics(df: pd.DataFrame, command: str, columns: Optional[List[str]] = None, limit: int = 5):
    subset = df[df.get("command") == command]
    if subset.empty:
        print(f"No rows for {command}.")
        return
    if columns is None:
        columns = [c for c in subset.columns if c.startswith("throughput") or c.startswith("latency")]
    available = [c for c in columns if c in subset.columns]
    display(subset[["artifact", "mode"] + available].head(limit))

In [ ]:
overhead_df = summarize_suite("overhead", limit=15)
if overhead_df.empty:
    print("No overhead artifacts found. Run the suite and re-run this cell.")
else:
    preview_metrics(overhead_df.assign(artifact=overhead_df['artifact']), command="lb-client")
    plot_throughput(overhead_df, command="lb-client", title="Overhead suite – lb-client throughput")

flow_accuracy_df = summarize_suite("flow_accuracy", limit=10)
if flow_accuracy_df.empty:
    print("No flow_accuracy artifacts yet.")
else:
    preview_metrics(flow_accuracy_df.assign(artifact=flow_accuracy_df['artifact']), command="lb-client", columns=["latency_us_p95", "latency_us_p99"])

false_sharing_df = summarize_suite("false_sharing", limit=10)
if false_sharing_df.empty:
    print("No false_sharing artifacts yet.")
else:
    preview_metrics(false_sharing_df.assign(artifact=false_sharing_df['artifact']), command="lb-client")
    plot_throughput(false_sharing_df, command="lb-client", title="False sharing – lb-client throughput")